In [104]:
from pyspark.sql import SparkSession

spark = SparkSession\
.builder\
.master("yarn")\
.appName("assignment-week7")\
.enableHiveSupport()\
.config("spark.sql.warehouse.dir","/user/itv009490/warehouse")\
.getOrCreate()

spark

In [2]:
!hdfs dfs -cat /public/trendytech/datasets/cust_transf.csv | head

1001,2023-05-15,1001,49.99
1002,2023-05-16,1002,29.99
1003,2023-05-17,1003,39.99
1004,2023-05-18,1004,19.99
1005,2023-05-19,1005,24.99
1001,2023-05-20,1002,29.99
1002,2023-05-21,1003,39.99
1003,2023-05-22,1004,19.99
1004,2023-05-23,1005,24.99
1005,2023-05-24,1001,49.99
cat: Unable to write to output stream.


In [15]:
schema = 'customer_id long, purchase_date date, product_id long, amount float'

In [16]:
df = spark.read.format("csv").\
schema(schema).\
load("/public/trendytech/datasets/cust_transf.csv")

In [17]:
df.show()

+-----------+-------------+----------+------+
|customer_id|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [11]:
spark.sql("""create database assignment_7""").show()

++
||
++
++



In [12]:
df.write.saveAsTable("assignment_7.products")

In [18]:
start_date="2023-05-01"
end_date="2023-06-08"

In [21]:
filtered_df = df.filter((df["purchase_date"]>=start_date) & (df["purchase_date"]<=end_date))

In [23]:
filtered_cache = filtered_df.cache()

In [26]:
revenue_df = filtered_cache.groupBy("product_id").sum("amount").withColumnRenamed("sum(amount)","revenue")

In [27]:
revenue_df.sort("revenue",ascending=False).show()

+----------+--------------------+
|product_id|             revenue|
+----------+--------------------+
|      1003| 5.725592484315491E8|
|      1001| 5.566826598912048E8|
|      1002| 4.293836211229706E8|
|      1004| 2.862080211229706E8|
|      1005|2.7828563865119934E8|
|      1015|   12537.91035079956|
|      1014|   11492.91035079956|
|      1013|   10447.91035079956|
|      1012|    9402.91035079956|
|      1011|    8357.91035079956|
|      1010|   7312.910350799561|
|      1009|   6267.909952163696|
|      1008|   5222.909952163696|
|      1007|   4177.909952163696|
|      1006|  3132.9099521636963|
+----------+--------------------+



In [60]:
cust_df = filtered_df.groupBy("customer_id").sum("amount").withColumnRenamed("sum(amount)","revenue")
cust_df.sort("revenue",ascending=False).limit(10).show()

+-----------+--------------------+
|customer_id|             revenue|
+-----------+--------------------+
|       1001| 3.180884683165741E8|
|       1004| 3.101342652822876E8|
|       1005| 2.624090592137146E8|
|       1003| 2.146838592137146E8|
|       1002| 2.067296592137146E8|
|       1011|1.2724374341640854E8|
|       1006| 1.272385184160099E8|
|       1012|1.1133638841640854E8|
|       1007| 1.113311634160099E8|
|       1013| 9.542903341640854E7|
+-----------+--------------------+



In [45]:
spark.sql("""create database itv009490_assingment_ext""")

""


In [64]:
spark.sql("""drop table itv009490_assingment_ext.customer_transactions_ext""")

""


In [65]:
spark.sql("""create table itv009490_assingment_ext.customer_transactions_ext 
          (customer_id long,purchase_date string,product_id integer,transaction_amount double) USING csv  location
          '/public/trendytech/datasets/cust_transf.csv'
          """)

""


In [56]:
!hdfs dfs -ls /user/itv009490/warehouse/itv009490_assingment_ext.db

In [66]:
spark.sql("""desc extended customer_transactions_ext""").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|         customer_id|              bigint|   null|
|       purchase_date|              string|   null|
|          product_id|                 int|   null|
|  transaction_amount|              double|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|itv009490_assingm...|       |
|               Table|customer_transact...|       |
|               Owner|           itv009490|       |
|        Created Time|Sun Nov 26 13:41:...|       |
|         Last Access|Wed Dec 31 19:00:...|       |
|          Created By|         Spark 2.4.7|       |
|                Type|            EXTERNAL|       |
|            Provider|                 csv|       |
|    Table Properties|[transient_lastDd...|       |
|            Location|hdfs://m01.itvers...|       |
|       Serd

In [69]:
spark.sql(f"""select product_id, sum(transaction_amount) as revenue from itv009490_assingment_ext.customer_transactions_ext 
where purchase_date>='2023-05-01'AND purchase_date<='2023-06-30'
group by product_id order by revenue desc limit 10
""").show()

+----------+-------------------+
|product_id|            revenue|
+----------+-------------------+
|      1001|8.747870076028482E8|
|      1003|6.997946075949881E8|
|      1002|5.248022075897805E8|
|      1005|4.373060075933379E8|
|      1004|3.498098075985675E8|
|      1015| 12537.909999999963|
|      1014| 11492.909999999963|
|      1013| 10447.909999999963|
|      1012|  9402.909999999965|
|      1011|  8357.909999999967|
+----------+-------------------+



In [71]:
spark.sql("""cache table itv009490_assingment_ext.customer_transactions_ext""")

""


In [72]:
spark.sql(f"""select product_id, sum(transaction_amount) as revenue from itv009490_assingment_ext.customer_transactions_ext 
where purchase_date>='2023-05-01'AND purchase_date<='2023-06-30'
group by product_id order by revenue desc limit 10
""").show()

+----------+--------------------+
|product_id|             revenue|
+----------+--------------------+
|      1001| 8.747870076028483E8|
|      1003| 6.997946075949881E8|
|      1002| 5.248022075897805E8|
|      1005| 4.373060075933379E8|
|      1004|3.4980980759856737E8|
|      1015|  12537.909999999963|
|      1014|  11492.909999999963|
|      1013|  10447.909999999963|
|      1012|   9402.909999999965|
|      1011|   8357.909999999967|
+----------+--------------------+



In [74]:
spark.sql(f"""select customer_id, sum(transaction_amount) as cust_transactions from itv009490_assingment_ext.customer_transactions_ext 
where purchase_date>='2023-05-01'AND purchase_date<='2023-06-30'
group by customer_id order by cust_transactions desc limit 10
""").show()

+-----------+--------------------+
|customer_id|   cust_transactions|
+-----------+--------------------+
|       1001| 3.180884580005336E8|
|       1004| 3.101342580008687E8|
|       1005|2.6240905800151232E8|
|       1003|2.1468385800145328E8|
|       1002| 2.067296580014408E8|
|       1011|1.9086143271084768E8|
|       1006|1.9085620771084768E8|
|       1015|1.6700301271081638E8|
|       1010|1.6699778771081635E8|
|       1014|1.5109356771079004E8|
+-----------+--------------------+



In [75]:
spark.sql("""clear cache""")

""


In [76]:
spark.sql("""select month(purchase_date) from  itv009490_assingment_ext.customer_transactions_ext""").show()

+----------------------------------+
|month(CAST(purchase_date AS DATE))|
+----------------------------------+
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 5|
|                                 6|
|                                 6|
|                                 6|
+----------------------------------+
only showing top 20 rows



In [86]:
spark.sql("""select distinct customer_id from (select customer_id,cnt from (select customer_id,count(customer_id)  over(partition by month(purchase_date)) as cnt from
itv009490_assingment_ext.customer_transactions_ext)
where cnt >1 order by cnt desc) limit 10
""").show()

+-----------+
|customer_id|
+-----------+
|       1002|
|       1009|
|       1013|
|       1005|
|       1015|
|       1008|
|       1004|
|       1003|
|       1010|
|       1012|
+-----------+



In [105]:
from pyspark.sql.functions import year,month
from pyspark.sql.functions import countDistinct

In [106]:
cust_schema='customer_id long,purchase_date date,product_id integer,transaction_amount double'

In [107]:
transactions_df=spark.read\
.format("csv")\
.schema(cust_schema)\
.load("/public/trendytech/datasets/cust_transf.csv")

In [108]:
new_df=transactions_df.withColumn("purchase_year",year("purchase_date")).withColumn("purchase_month",month("purchase_date"))

In [109]:
new_df.show()

+-----------+-------------+----------+------------------+-------------+--------------+
|customer_id|purchase_date|product_id|transaction_amount|purchase_year|purchase_month|
+-----------+-------------+----------+------------------+-------------+--------------+
|       1001|   2023-05-15|      1001|             49.99|         2023|             5|
|       1002|   2023-05-16|      1002|             29.99|         2023|             5|
|       1003|   2023-05-17|      1003|             39.99|         2023|             5|
|       1004|   2023-05-18|      1004|             19.99|         2023|             5|
|       1005|   2023-05-19|      1005|             24.99|         2023|             5|
|       1001|   2023-05-20|      1002|             29.99|         2023|             5|
|       1002|   2023-05-21|      1003|             39.99|         2023|             5|
|       1003|   2023-05-22|      1004|             19.99|         2023|             5|
|       1004|   2023-05-23|      1005|     

In [110]:
customer_month_counts=new_df.groupBy("customer_id","purchase_year","purchase_month").agg(countDistinct("purchase_month").alias("distinct_months"))
customer_month_counts.show()

+-----------+-------------+--------------+---------------+
|customer_id|purchase_year|purchase_month|distinct_months|
+-----------+-------------+--------------+---------------+
|       1011|         2023|             5|              1|
|       1002|         2023|             5|              1|
|       1004|         2023|             6|              1|
|       1014|         2023|             5|              1|
|       1004|         2023|             5|              1|
|       1009|         2023|             6|              1|
|       1013|         2023|             6|              1|
|       1002|         2023|             6|              1|
|       1013|         2023|             5|              1|
|       1001|         2023|             6|              1|
|       1007|         2023|             6|              1|
|       1007|         2023|             5|              1|
|       1015|         2023|             6|              1|
|       1006|         2023|             5|              

In [111]:
regular_customers=customer_month_counts.filter("distinct_months=1")\
.groupBy("customer_id").count()\
.orderBy("count",ascending=False).limit(10)
regular_customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1011|    2|
|       1007|    2|
|       1002|    2|
|       1005|    2|
|       1003|    2|
|       1013|    2|
|       1001|    2|
|       1004|    2|
|       1008|    2|
|       1010|    2|
+-----------+-----+



In [112]:
#persist

customer_month_counts=new_df.groupBy("customer_id","purchase_year","purchase_month")\
.agg(countDistinct("purchase_month").alias("distinct_months")).persist()

In [113]:
regular_customers=customer_month_counts.filter("distinct_months=1")\
.groupBy("customer_id").count()\
.orderBy("count",ascending=False).limit(10)
regular_customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1005|    2|
|       1004|    2|
|       1003|    2|
|       1014|    2|
|       1012|    2|
|       1008|    2|
|       1015|    2|
|       1001|    2|
|       1002|    2|
|       1006|    2|
+-----------+-----+



In [114]:
regular_customers=customer_month_counts.filter("distinct_months=1")\
.groupBy("customer_id").count()\
.orderBy("count",ascending=False).limit(10).show()


+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1005|    2|
|       1013|    2|
|       1004|    2|
|       1003|    2|
|       1002|    2|
|       1014|    2|
|       1006|    2|
|       1001|    2|
|       1008|    2|
|       1012|    2|
+-----------+-----+



In [115]:
spark.sql("""clear cache""")

""


In [116]:
#cache

customer_month_counts=new_df.groupBy("customer_id","purchase_year","purchase_month")\
.agg(countDistinct("purchase_month").alias("distinct_months")).cache()

In [117]:
regular_customers=customer_month_counts.filter("distinct_months=1")\
.groupBy("customer_id").count()\
.orderBy("count",ascending=False).limit(10)
regular_customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1014|    2|
|       1005|    2|
|       1012|    2|
|       1010|    2|
|       1006|    2|
|       1015|    2|
|       1011|    2|
|       1007|    2|
|       1009|    2|
|       1008|    2|
+-----------+-----+



In [118]:
regular_customers=customer_month_counts.filter("distinct_months=1")\
.groupBy("customer_id").count()\
.orderBy("count",ascending=False).limit(10).show()


+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1010|    2|
|       1014|    2|
|       1012|    2|
|       1005|    2|
|       1008|    2|
|       1006|    2|
|       1015|    2|
|       1013|    2|
|       1004|    2|
|       1002|    2|
+-----------+-----+

